In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
results = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
seeds = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneySeeds.csv')
rank = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MMasseyOrdinals.csv')
teams = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MTeams.csv')

print(results[1150:].head())
print(seeds.head())
print(rank.head())

In [ ]:
rank = rank[rank["RankingDayNum"] == 133]
rank = rank[rank["SystemName"] == "EBP"]
rank = rank.pivot(index="TeamID", columns="Season", values="OrdinalRank")

rank.head()

In [ ]:
W_seed = []
L_seed = []

seed_key = {}

for i in range(len(seeds)):
    s = seeds.loc[i,"Seed"]
    k = seeds.loc[i,"TeamID"]
    seed_key[k] = s

for i in range(len(results) - 500):
    i = i + 500
    W_seed.append(int(seed_key[results.loc[i,"WTeamID"]][1:3]))
    L_seed.append(int(seed_key[results.loc[i,"LTeamID"]][1:3]))
    
#print(seed_key)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sbn

game = [(W_seed[i],L_seed[i]) for i in range(len(W_seed))]

n = len(game)

# chart is total wins by higher seeded team (low seed, high seed)
wins = np.zeros(shape = (18,18))
chart = np.zeros(shape = (18,18))

# total times teams have played in form (low seed, high seed)
total = np.zeros(shape = (18,18))

for x,y in game:
    wins[min(x,y)][max(x,y)] += int(x<y)
    total[min(x,y)][max(x,y)] += 1
    
for x in range(18):
    for y in range(18):
        t = 1
        if total[x][y] != 0:
            t = total[x][y]
        chart[x][y] = wins[x][y] / t
        
        if x + y == 17 and  x > 0 and x < 9:
            print(x,y,chart[x][y])

higher_seed_win_prob = chart.copy()
sample_size = total.copy()
sbn.heatmap(chart, cmap= "Reds")


#print(total)
#print(wins)
#print(chart)

In [ ]:
def f(t1,t2):
    w = 0
    s = 0
    
    #upset meter makes upsets X% less likely
    upset_meter = 1
    for x in [-1,0,1]:
        for y in [-1,0,1]:
            w += (1 / (1 + np.sqrt(x**2 + y**2))) * wins[t1 + x][t2 + y]
            s += (1 / (1 + np.sqrt(x**2 + y**2))) * sample_size[t1 + x][t2 + y]

    return (w / s) * (1+upset_meter)

In [ ]:
import random as r

class Team:
    
    def __init__(self, seed, name):
        self.seed = seed
        self.name = name
    
    def __init__(self, teamID):
        self.seed = int(seeds.loc[seeds['TeamID'] == teamID]['Seed'].values[0][1:3])
        self.name = teams.loc[teams['TeamID'] == teamID]['TeamName'].values[0]
    
    def __init__(self, teamID, seed):
        self.name = teams.loc[teams['TeamID'] == teamID]['TeamName'].values[0]
        self.seed = seed
        
    def __repr__(self):
        return self.name
        
    

class Game:
    
    def __init__(self, t1, t2):
        self.t1 = t1
        self.t2 = t2
        
    def pred(self):      
        thresh = f(min(self.t1.seed,self.t2.seed),max(self.t1.seed,self.t2.seed))
        if r.random() < thresh:
            return self.t1
        else:
            return self.t2
        
    
    
class Bracket:
    
    def __init__(self, ids):
        seed_numbers = [1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15, 
                        1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15, 
                        1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15, 
                        1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]
        self.ids = ids
        self.bracket = [Team(teamID=self.ids[i], seed=seed_numbers[i]) for i in range(64)]
        
 
    def play(self, sims=1, silent=False):
        winners = []
        for n in range(sims):
            if not silent:
                print()
                print("Simulating tournament: ", n+1)
            bracket = self.bracket.copy()
            results = [self.bracket.copy()]
            while(len(bracket) > 1):
                new_round = []
                for i in range(len(bracket)//2):
                    g = Game(bracket[2*i], bracket[2*i+1])
                    new_round.append(g.pred())
                if not silent:
                    print(new_round)
                results.append(new_round.copy())
                bracket = new_round.copy()
            winners.append(bracket[0])
        
        if not silent:
            print()
            print("Winner: ")
        return winners
    

In [ ]:
from collections import Counter
team_IDs = [1211, 1209, 1239, 1235, 1222, 1448, 1344, 1436, 1388, 1274,
                    1437, 1449, 1425, 1362, 1124, 1389, 1112, 1313, 1277, 1361,
                    1163, 1374, 1417, 1151, 1400, 1172, 1345, 1159, 1129, 1260, 
                    1181, 1240, 1246, 1460, 1371, 1323, 1234, 1439, 1116, 1308,
                    1326, 1231, 1397, 1286, 1161, 1395, 1120, 1174, 1242, 1136, 
                    1116, 1266, 1261, 1353, 1228, 1343, 1314, 1276, 1403, 1255,
                    1104, 1272, 1458, 1253]
b = Bracket(team_IDs)
#X = b.play(sims=1000, silent=True)
#c = Counter(X)

#print(c.most_common())

print(b.play())

In [ ]:
# for jake code
# f is a function that makes new percentages
def f(t1,t2):
    w = 0
    s = 0
    
    #upset meter makes upsets X% less likely
    upset_meter = 1
    for x in [-1,0,1]:
        for y in [-1,0,1]:
            w += (1 / (1 + np.sqrt(x**2 + y**2))) * wins[t1 + x][t2 + y]
            s += (1 / (1 + np.sqrt(x**2 + y**2))) * sample_size[t1 + x][t2 + y]

    return (w / s) * (1+upset_meter)


function_chart = np.zeros(shape = (18,18))
for x in range(1,17):
    for y in range(x+1,17):
        function_chart[x][y] = f(x,y)
     

In [ ]:
# Recursive Bracket Cell with dictionary instead of an array
bracket = [[1211,1], [1209,16], [1239,8], [1235,9], [1222,5], [1448,12], [1344,4], [1436,13], [1388,6], [1274,11], [1437,3], [1449,14], [1425,7], [1362,10], [1124,2], [1389,15],
            [1112,1], [1313,16], [1277,8], [1361,9], [1163,5], [1374,12], [1417,4], [1151,13], [1400,6], [1172,11], [1345,3], [1159,14], [1129,7], [1260,10], [1181,2], [1240,15],
            [1246,1], [1460,16], [1371,8], [1323,9], [1234,5], [1439,12], [1116,4], [1308,13], [1326,6], [1231,11], [1397,3], [1286,14], [1161,7], [1395,10], [1120,2], [1174,15],
            [1242,1], [1136,16], [1116,8], [1266,9], [1261,5], [1353,12], [1228,4], [1343,13], [1314,6], [1276,11], [1403,3], [1255,14], [1104,7], [1272,10], [1458,2], [1253,15]]

def round(bracket):
    newbracket = []
    nameprint = []
    length = int(len(bracket) / 2)
    for y in range(0,length):
        rand = r.random()
        teamone = min(bracket[2 * y], bracket[(2 * y) + 1])
        teamtwo = max(bracket[2 * y], bracket[(2 * y) + 1])
        if rand < (function_chart[teamone[1]][teamtwo[1]]):
            newbracket.append(teamone)
            name = teams.loc[teams['TeamID'] == teamone[0]]['TeamName'].values[0]
            nameprint.append(name)
        else:
            newbracket.append(teamtwo)
            name = teams.loc[teams['TeamID'] == teamtwo[0]]['TeamName'].values[0]
            nameprint.append(name)
    print(nameprint)
    if len(newbracket) > 1:
        round(newbracket.copy())
        
round(bracket)